# linking

> WRDS Concordances

Since WRDS is a subscription service, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.linking

In [ ]:
#| exports
import pandas as pd

import pandasmore as pdm
from finsets.wrds import wrds_api

## Merging CRSP and COMPUSTAT

Official documentation for this merge is [here](https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/linking-databases/linking-crsp-and-compustat/)

Relevant information from that site:

- Because TICKERs and CUSIPs change over time, most datasets use permanent identifiers to indicate the same securities and companies. For example, CRSP employs PERMNO to track stocks, Compustat uses GVKEY to follow companies, and a combination of GVKEY and IID is used to track securities. As indicated in the preceding table, both PERMNO and GVKEY (+IID) remain the same, regardless of changes in TICKER, CUSIP, and company names.

- The CRSP/Compustat Merged Database (CCM) is comprised of CRSP and Compustat data, together with the link and link-history references between these two databases. The key product of CCM is a permanent identifier linking table (PERMNO to GVKEY+IID), though CCM is often wrongly mistaken as the merged product of CRSP stock market data with Compustat accounting data. From an end user’s perspective, CCM only adds a link of PERMNO and PERMCO to the Compustat database, so that Compustat items can be searched by CRSP identifiers.


In [ ]:
#| export
def gvkey_permno_m(wrds_username: str=None,
                   obs_limit: int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """CRSP Monthly ids, with gvkeys"""

    limit_clause = f"LIMIT {obs_limit}" if obs_limit is not None else ""
    sql_string=f"""SELECT a.date, a.permno, c.gvkey
                  FROM crsp.msf a
                  INNER JOIN crsp.msenames b ON a.permno = b.permno
                                             AND a.date BETWEEN b.namedt AND b.nameendt 
                  INNER JOIN crsp.ccmxpf_lnkhist c ON a.permno = c.lpermno 
                                                   AND c.linktype IN ('LU','LC') AND c.linkprim IN ('P','C')
                                                   AND a.date BETWEEN c.linkdt AND COALESCE(c.linkenddt, CURRENT_DATE)
                    {limit_clause}                                   
                """
    
    df = wrds_api.download(sql_string, wrds_username)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='date', freq='M',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Mdate','gvkey']].copy()  

In [ ]:
#| eval: false
df = gvkey_permno_m(obs_limit=100)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Mdate,gvkey
0,10000,1986-01,013007
1,10000,1986-02,013007
2,10000,1986-03,013007
3,10000,1986-04,013007
4,10000,1986-05,013007
...,...,...,...
95,10001,1992-07,012994
96,10001,1992-08,012994
97,10001,1992-09,012994
98,10001,1992-10,012994


In [ ]:
#| export
def gvkey_permno_a(wrds_username: str=None,
                   obs_limit: int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """qvkey to permno correspondence at the annual frequency. As done by CCM."""

    limit_clause = f"LIMIT {obs_limit}" if obs_limit is not None else ""
    sql_string=f"""SELECT a.datadate, a.gvkey , b.lpermno as permno
                    FROM comp.funda a
                    INNER JOIN crsp.ccmxpf_lnkhist  b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') AND b.linkprim IN ('P','C')
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
                    {limit_clause}            
                """
    
    df = wrds_api.download(sql_string, wrds_username)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='datadate', freq='A',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Adate','gvkey']].copy()  

In [ ]:
#| eval: false
df = gvkey_permno_a(obs_limit=100)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Adate,gvkey
0,10015,1983,001001
1,10015,1984,001001
2,10015,1985,001001
3,10023,1972,001002
4,10031,1983,001003
...,...,...,...
95,61903,1977,001005
96,61903,1978,001005
97,61903,1979,001005
98,61903,1980,001005


In [ ]:
#| export
def gvkey_permno_q(wrds_username: str=None,
                   obs_limit : int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """qvkey to permno correspondence at the quarterly frequency. As done by CCM."""
    
    limit_clause = f"LIMIT {obs_limit}" if obs_limit is not None else ""
    sql_string=f"""SELECT a.datadate, a.gvkey , b.lpermno as permno
                    FROM comp.fundq a
                    INNER JOIN crsp.ccmxpf_lnkhist  b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') AND b.linkprim IN ('P','C')
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'            
                    {limit_clause}
                """
    
    df = wrds_api.download(sql_string, wrds_username)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='datadate', freq='Q',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Qdate','gvkey']].copy()    

In [ ]:
#| eval: false
df = gvkey_permno_q(obs_limit=100)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Qdate,gvkey
0,10006,1967Q4,001010
1,10006,1968Q1,001010
2,10015,1983Q3,001001
3,10015,1983Q4,001001
4,10015,1984Q1,001001
...,...,...,...
95,54594,1979Q1,001004
96,54594,1979Q2,001004
97,54594,1979Q3,001004
98,54594,1979Q4,001004


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()